In [1]:
import time

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

import numpy as np

/home/shashwati/anaconda3/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [3]:
# function returns the data in the right format
def get_data():
    dataset = np.genfromtxt("connect-4.csv", dtype='str', delimiter=",")
    
    preX = dataset[:,0:42]
    preY = dataset[:,42]
    
    X = np.zeros(preX.shape)
    
    for i, row in enumerate(preX):
        for j, col in enumerate(row):
            if col == 'x':
                X[i,j] = 1.0
            if col == 'o':
                X[i,j] = -1.0
            if col == 'b':
                X[i,j] = 0.0
    
    
    encoder = LabelEncoder()
    # code: 0 - draw; 1 - loss; 2 -win
    encoded_Y = encoder.fit_transform(preY)
    
    
    # splitting the dataset into 80% training and 20% test data set
    train, test, label_train, label_test = \
            train_test_split(X, encoded_Y,test_size = 0.2)
    
    return train, label_train, test, label_test

In [4]:
from sklearn.tree._tree import TREE_LEAF

def prune_index(inner_tree, index, threshold):
    if inner_tree.value[index].min() < threshold:
        # turn node into a leaf by "unlinking" its children
        inner_tree.children_left[index] = TREE_LEAF
        inner_tree.children_right[index] = TREE_LEAF
    # if there are shildren, visit them as well
    if inner_tree.children_left[index] != TREE_LEAF:
        prune_index(inner_tree, inner_tree.children_left[index], threshold)
        prune_index(inner_tree, inner_tree.children_right[index], threshold)

In [5]:
# builds the decision tree of depth 12
def decision_tree(train, label):
    dt = DecisionTreeClassifier(max_depth = 12, min_samples_leaf=100)
    dt.fit(train, label)
    prune_index(dt.tree_, 0, 5)
    end = time.time()
    return dt

In [6]:
# builds the neural network for a given class
def neural_network(class_data):
    num_train = []
    num_label = []
    for x in class_data:
        num_train.append(x[0])
        num_label.append(x[1])
    
    num_train = np.array(num_train)
    num_label = np.array(num_label)
    
    # converting categorical variable into numerical values
    encoder = LabelEncoder()
    encoder.fit(num_label)

    # code: 0 - draw; 1 - loss; 2 -win
    encoded_Y = encoder.transform(num_label)
    final_label = np_utils.to_categorical(encoded_Y, 3)
    
    out = final_label.shape[1]
    print(out, final_label.shape, num_train.shape)
    model = Sequential()
    model.add(Dense(8, input_dim=42, activation='relu'))
    model.add(Dense(out, activation='softmax'))
    model.compile(loss='categorical_crossentropy', \
                  optimizer='adam', metrics=['accuracy'])
    model.fit(num_train, final_label, epochs=5, batch_size=5)
    return model 

In [7]:
# builds the neural shrub
def neural_shrubs(tree, train, label):
    train = np.array(train)
    label = np.array(label)
  
    # leave_id: index of the leaf that cantains the instance
    leave_id = tree.apply(train) 

    classes = dict()

    for x in range(len(train)):
        leaf = leave_id[x]
        
        # Gets the class for each leaf
        #.value: contains value of all the tree nodes
        #.value[leaf]: returns the value of the leaf
        #idx = tree.tree_.value[leaf][0][0]
        
        # insert the instance into the class
        if leaf in classes.keys():
            classes[leaf].append([train[x], label[x]])
        else:
            classes[leaf] = [[train[x], label[x]]]
    
    # stores the neural network for each class
    nn_models = dict()
    
    #stores the max time taken to build a neural network
    max_time = 0;
    for key in classes.keys():            
        start = time.time()
        model = neural_network(classes[key])
        end = time.time()
        
        time_taken = end - start
        if max_time < time_taken:
            max_time = time_taken
        
        nn_models[key] = model
   
    # returns a neural network for each class and the max 
    # time taken to build the neural network
    return nn_models, max_time

In [8]:
# The algorithm to build the neural shrub
train, train_label, test, test_label = get_data()

dt_start = time.time()
tree = decision_tree(train, train_label)
dt_end = time.time()

shrubs, max_time = neural_shrubs(tree, train, train_label)

3 (140, 3) (140, 42)
Epoch 1/5
140/140 [==============================] - 0s 3ms/step - loss: 1.0481 - acc: 0.3429
Epoch 2/5
140/140 [==============================] - 0s 449us/step - loss: 1.0055 - acc: 0.5071
Epoch 3/5
140/140 [==============================] - 0s 530us/step - loss: 0.9700 - acc: 0.5500
Epoch 4/5
140/140 [==============================] - 0s 422us/step - loss: 0.9404 - acc: 0.6000
Epoch 5/5
140/140 [==============================] - 0s 494us/step - loss: 0.9139 - acc: 0.6143
3 (300, 3) (300, 42)
Epoch 1/5
300/300 [==============================] - 1s 2ms/step - loss: 0.7954 - acc: 0.7167
Epoch 2/5
300/300 [==============================] - 0s 510us/step - loss: 0.6442 - acc: 0.7233
Epoch 3/5
300/300 [==============================] - 0s 434us/step - loss: 0.5474 - acc: 0.7367
Epoch 4/5
300/300 [==============================] - 0s 404us/step - loss: 0.4899 - acc: 0.7467
Epoch 5/5
300/300 [==============================] - 0s 387us/step - loss: 0.4512 - acc: 0.7600
3 

101/101 [==============================] - 0s 637us/step - loss: 0.5340 - acc: 0.9604
Epoch 4/5
101/101 [==============================] - 0s 650us/step - loss: 0.4303 - acc: 0.9604
Epoch 5/5
101/101 [==============================] - 0s 648us/step - loss: 0.3530 - acc: 0.9604
3 (1173, 3) (1173, 42)
Epoch 1/5
1173/1173 [==============================] - 2s 1ms/step - loss: 1.1397 - acc: 0.3478
Epoch 2/5
1173/1173 [==============================] - 1s 501us/step - loss: 1.0087 - acc: 0.4902
Epoch 3/5
1173/1173 [==============================] - 1s 579us/step - loss: 0.9554 - acc: 0.5610
Epoch 4/5
1173/1173 [==============================] - 1s 500us/step - loss: 0.9168 - acc: 0.6070
Epoch 5/5
1173/1173 [==============================] - 1s 494us/step - loss: 0.8829 - acc: 0.6317
3 (222, 3) (222, 42)
Epoch 1/5
222/222 [==============================] - 1s 4ms/step - loss: 1.1542 - acc: 0.3063
Epoch 2/5
222/222 [==============================] - 0s 519us/step - loss: 1.1292 - acc: 0.3198


191/191 [==============================] - 0s 481us/step - loss: 0.9669 - acc: 0.5550
3 (372, 3) (372, 42)
Epoch 1/5
372/372 [==============================] - 1s 3ms/step - loss: 1.0213 - acc: 0.4919
Epoch 2/5
372/372 [==============================] - 0s 500us/step - loss: 0.9436 - acc: 0.5591
Epoch 3/5
372/372 [==============================] - 0s 553us/step - loss: 0.9125 - acc: 0.5591
Epoch 4/5
372/372 [==============================] - 0s 482us/step - loss: 0.8935 - acc: 0.5591
Epoch 5/5
372/372 [==============================] - 0s 562us/step - loss: 0.8783 - acc: 0.5699
3 (211, 3) (211, 42)
Epoch 1/5
211/211 [==============================] - 1s 5ms/step - loss: 1.5756 - acc: 0.1422
Epoch 2/5
211/211 [==============================] - 0s 484us/step - loss: 1.3007 - acc: 0.1991
Epoch 3/5
211/211 [==============================] - 0s 530us/step - loss: 1.1328 - acc: 0.3697
Epoch 4/5
211/211 [==============================] - 0s 608us/step - loss: 1.0088 - acc: 0.4976
Epoch 5/5
21

133/133 [==============================] - 0s 581us/step - loss: 1.0283 - acc: 0.4962
Epoch 3/5
133/133 [==============================] - 0s 638us/step - loss: 0.9305 - acc: 0.6316
Epoch 4/5
133/133 [==============================] - 0s 580us/step - loss: 0.8591 - acc: 0.6767
Epoch 5/5
133/133 [==============================] - 0s 675us/step - loss: 0.8077 - acc: 0.7068
3 (160, 3) (160, 42)
Epoch 1/5
160/160 [==============================] - 1s 9ms/step - loss: 0.9935 - acc: 0.5375
Epoch 2/5
160/160 [==============================] - 0s 730us/step - loss: 0.8791 - acc: 0.6688
Epoch 3/5
160/160 [==============================] - 0s 651us/step - loss: 0.8127 - acc: 0.7125
Epoch 4/5
160/160 [==============================] - 0s 595us/step - loss: 0.7727 - acc: 0.7188
Epoch 5/5
160/160 [==============================] - 0s 661us/step - loss: 0.7495 - acc: 0.7188
3 (110, 3) (110, 42)
Epoch 1/5
110/110 [==============================] - 1s 12ms/step - loss: 1.1010 - acc: 0.5091
Epoch 2/5
1

100/100 [==============================] - 0s 648us/step - loss: 1.0375 - acc: 0.4700
Epoch 4/5
100/100 [==============================] - 0s 723us/step - loss: 0.9919 - acc: 0.5800
Epoch 5/5
100/100 [==============================] - 0s 806us/step - loss: 0.9490 - acc: 0.6300
3 (188, 3) (188, 42)
Epoch 1/5
188/188 [==============================] - 2s 9ms/step - loss: 0.9975 - acc: 0.5000
Epoch 2/5
188/188 [==============================] - 0s 692us/step - loss: 0.8752 - acc: 0.6968
Epoch 3/5
188/188 [==============================] - 0s 672us/step - loss: 0.7911 - acc: 0.6968
Epoch 4/5
188/188 [==============================] - 0s 623us/step - loss: 0.7428 - acc: 0.6968
Epoch 5/5
188/188 [==============================] - 0s 723us/step - loss: 0.7130 - acc: 0.6915
3 (233, 3) (233, 42)
Epoch 1/5
233/233 [==============================] - 2s 8ms/step - loss: 1.6986 - acc: 0.0644
Epoch 2/5
233/233 [==============================] - 0s 887us/step - loss: 1.2401 - acc: 0.0944
Epoch 3/5
23

...

177/177 [==============================] - 5s 26ms/step - loss: 1.1296 - acc: 0.2881
Epoch 2/5
177/177 [==============================] - 0s 1ms/step - loss: 1.0096 - acc: 0.5198
Epoch 3/5
177/177 [==============================] - ETA: 0s - loss: 0.9265 - acc: 0.614 - 0s 1ms/step - loss: 0.9303 - acc: 0.6328
Epoch 4/5
177/177 [==============================] - 0s 1ms/step - loss: 0.8683 - acc: 0.6949
Epoch 5/5
177/177 [==============================] - 0s 1ms/step - loss: 0.8214 - acc: 0.7232
3 (165, 3) (165, 42)
Epoch 1/5
165/165 [==============================] - 5s 28ms/step - loss: 0.9528 - acc: 0.5515
Epoch 2/5
165/165 [==============================] - 0s 1ms/step - loss: 0.8032 - acc: 0.7455
Epoch 3/5
165/165 [==============================] - 0s 1ms/step - loss: 0.6959 - acc: 0.7758
Epoch 4/5
165/165 [==============================] - 0s 1ms/step - loss: 0.6437 - acc: 0.7758
Epoch 5/5
165/165 [==============================] - 0s 1ms/step - loss: 0.6127 - acc: 0.7758
3 (112, 3

Epoch 4/5
114/114 [==============================] - 0s 1ms/step - loss: 1.1306 - acc: 0.3860
Epoch 5/5
114/114 [==============================] - 0s 1ms/step - loss: 1.1017 - acc: 0.4123
3 (166, 3) (166, 42)
Epoch 1/5
166/166 [==============================] - 5s 28ms/step - loss: 1.0899 - acc: 0.3735
Epoch 2/5
166/166 [==============================] - 0s 1ms/step - loss: 1.0391 - acc: 0.4157
Epoch 3/5
166/166 [==============================] - 0s 1ms/step - loss: 1.0042 - acc: 0.4157
Epoch 4/5
166/166 [==============================] - 0s 1ms/step - loss: 0.9804 - acc: 0.4217
Epoch 5/5
166/166 [==============================] - 0s 1ms/step - loss: 0.9634 - acc: 0.4217
3 (162, 3) (162, 42)
Epoch 1/5
162/162 [==============================] - 5s 28ms/step - loss: 1.0107 - acc: 0.5864
Epoch 2/5
162/162 [==============================] - 0s 1ms/step - loss: 0.9481 - acc: 0.6481
Epoch 3/5
162/162 [==============================] - 0s 1ms/step - loss: 0.9075 - acc: 0.6728
Epoch 4/5
162/16

142/142 [==============================] - 0s 1ms/step - loss: 1.1287 - acc: 0.3873
Epoch 3/5
142/142 [==============================] - 0s 1ms/step - loss: 1.0718 - acc: 0.4859
Epoch 4/5
142/142 [==============================] - 0s 1ms/step - loss: 1.0288 - acc: 0.5282
Epoch 5/5
142/142 [==============================] - 0s 1ms/step - loss: 0.9942 - acc: 0.5352
3 (194, 3) (194, 42)
Epoch 1/5
194/194 [==============================] - 5s 25ms/step - loss: 1.1756 - acc: 0.2835
Epoch 2/5
194/194 [==============================] - 0s 1ms/step - loss: 1.1058 - acc: 0.3918
Epoch 3/5
194/194 [==============================] - 0s 1ms/step - loss: 1.0724 - acc: 0.4588
Epoch 4/5
194/194 [==============================] - 0s 1ms/step - loss: 1.0422 - acc: 0.5000
Epoch 5/5
194/194 [==============================] - 0s 1ms/step - loss: 1.0159 - acc: 0.5361
3 (270, 3) (270, 42)
Epoch 1/5
270/270 [==============================] - 5s 18ms/step - loss: 1.4855 - acc: 0.1111
Epoch 2/5
270/270 [=======

169/169 [==============================] - 0s 1ms/step - loss: 0.9632 - acc: 0.5976
3 (195, 3) (195, 42)
Epoch 1/5
195/195 [==============================] - 5s 27ms/step - loss: 1.4463 - acc: 0.1282
Epoch 2/5
195/195 [==============================] - 0s 1ms/step - loss: 1.0856 - acc: 0.4256
Epoch 3/5
195/195 [==============================] - 0s 1ms/step - loss: 0.8484 - acc: 0.7333
Epoch 4/5
195/195 [==============================] - 0s 1ms/step - loss: 0.7269 - acc: 0.7692
Epoch 5/5
195/195 [==============================] - 0s 2ms/step - loss: 0.6710 - acc: 0.7744
3 (222, 3) (222, 42)
Epoch 1/5
222/222 [==============================] - 5s 24ms/step - loss: 1.1860 - acc: 0.1892
Epoch 2/5
222/222 [==============================] - 0s 1ms/step - loss: 0.8140 - acc: 0.7252
Epoch 3/5
222/222 [==============================] - 0s 1ms/step - loss: 0.5515 - acc: 0.9505
Epoch 4/5
222/222 [==============================] - 0s 1ms/step - loss: 0.3809 - acc: 0.9640
Epoch 5/5
222/222 [=======

158/158 [==============================] - 0s 1ms/step - loss: 1.0393 - acc: 0.4177
Epoch 4/5
158/158 [==============================] - 0s 1ms/step - loss: 1.0176 - acc: 0.4684
Epoch 5/5
158/158 [==============================] - 0s 1ms/step - loss: 1.0002 - acc: 0.4937


In [9]:
# predicts using the neural shrub
def neural_shrub_predict(tree, nn_model, test, label):
    label_test = np.array(label)
    test = np.array(test)
    
    #row - actual; col - pred
    confusion_matrix = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
    correct = 0
    
    for i in range(len(test)):
        x = test[i]
        pred_class = tree.apply([x])
        x = np.array([x])
        nn_model_class = nn_model[pred_class[0]]
        pred = np.argmax(nn_model_class.predict(x))

        confusion_matrix[label[i]][pred] = \
            confusion_matrix[label[i]][pred] + 1
        if pred == label[i]: correct = correct + 1

    acc_score = correct/len(test)
    
    return confusion_matrix, acc_score

In [10]:
# Predicting
cm, acc_score = neural_shrub_predict(tree, shrubs, test, test_label)
print("Confusion Matrix:\n\n", cm)

Confusion Matrix:

 [[  55  338  852]
 [ 136 1890 1364]
 [ 205 1287 7385]]


In [11]:
# function used to calcultate the metrics for each class
def metrics(cm, cls, size):
    cm = np.array(cm)
    tp = cm[cls][cls]
    fp = sum(cm[x, cls] for x in range(3))-cm[cls][cls]
    fn = sum(cm[cls, x] for x in range(3))-cm[cls][cls]
    tn = size - tp - fp - fn
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    fmeasure = 2*(precision*recall)/(precision + recall)
    accuracy = (tp + tn)/size
    
    return precision, recall, fmeasure, accuracy

In [12]:
# metrics for class 0 (draw)
precision0, recall0, f0, acc0 = metrics(cm, 0, len(test))
print("                Precision Recall F-measure Accuracy")
print("Class 0 (draw): ", round(precision0, 3), "  ", round(recall0, 3), \
      " ", round(f0, 3), "   ", round(acc0,3))

                Precision Recall F-measure Accuracy
Class 0 (draw):  0.139    0.044   0.067     0.887


In [13]:
# metrics for class 1 (lose)
precision1, recall1, f1, acc1 = metrics(cm, 1, len(test))
print("                Precision Recall F-measure Accuracy")
print("Class 1 (loss): ", round(precision1, 3), "  ", round(recall1, 3), \
      " ", round(f1, 3), "   ", round(acc1,3))

                Precision Recall F-measure Accuracy
Class 1 (loss):  0.538    0.558   0.547     0.769


In [14]:
# metrics for class 2 (win)
precision2, recall2, f2, acc2 = metrics(cm, 2, len(test))
print("                Precision Recall F-measure Accuracy")
print("Class 2 (win): ", round(precision2, 3), "  ", round(recall2, 3), \
      " ", round(f2, 3), "   ", round(acc2,3))

                Precision Recall F-measure Accuracy
Class 2 (win):  0.769    0.832   0.799     0.726


In [15]:
# average metrics
avg_p = (precision0 + precision1 + precision2)/3.0
avg_r = (recall0 + recall1 + recall2) / 3.0
avg_f = (f0 + f1 + f2) / 3.0
avg_a = (acc0 + acc1 + acc2)/ 3.0
print("        Precision Recall F-measure Accuracy")
print("Average: ", round(avg_p, 3), "  ", round(avg_r, 3), \
      " ", round(avg_f, 3), "   ", round(avg_a,3))

        Precision Recall F-measure Accuracy
Average:  0.482    0.478   0.471     0.794


In [16]:
# training time
total_time_taken = dt_end - dt_start + max_time
print("Training Time: %s sec" % round(total_time_taken, 5))

Training Time: 15.54949 sec


In [17]:
# Number of instances correctly classified
print("Accuracy_score: ", round(acc_score, 5))

Accuracy_score:  0.6905
